### 第2题 word2vec向量表示和文本分类（30分）

In [1]:
from sklearn.model_selection import train_test_split
import word2vec    #w2v包
import numpy as np

#review corpus: [reveiws,y]
y=[]
reviews=[]
with open("train.txt",'r',encoding='utf-8') as infile: # train.txt=>sentence list
    for line in infile:
        label, sentence = line.strip().split("\t")
        y.append(int(label))
        reviews.append(sentence)

x_train,x_test,y_train,y_test=train_test_split(reviews, y,test_size=0.2)


#sentence ->word list
def cleanText(corpus):  #输入corpus: sentence list
    corpus=[z.lower().replace('\n','').split() for z in corpus]
    # corpus=[z.replace('.','').split() for z in corpus]
    
    return corpus

x_train=cleanText(x_train)   # =>lowercase, word list
x_test=cleanText(x_test)


def claenagain(corpus):
    for wordlist in corpus:
        for word in wordlist:
            word = ''.join(filter(str.isalpha, word))
    return corpus


x_train=claenagain(x_train) 
x_test=claenagain(x_test)

In [2]:
###编写代码，采用自训练或者预训练方式获得word2vec词向量

from operator import mod
from gensim.models import KeyedVectors    
model=KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)
from gensim.models import Word2Vec
from gensim.models import KeyedVectors 
from gensim.test.utils import datapath, get_tmpfile   #word similarity package
from gensim.scripts.glove2word2vec import glove2word2vec

eg = model['fuck']
print(eg)

[ 1.04492188e-01 -3.12500000e-01  2.13867188e-01  3.08593750e-01
 -1.60156250e-01  9.42382812e-02 -1.60156250e-01 -3.22265625e-02
  1.69921875e-01  8.49609375e-02 -1.38671875e-01 -2.31445312e-01
 -6.49414062e-02  6.68945312e-02 -2.16796875e-01  1.82617188e-01
 -2.17773438e-01 -8.78906250e-03 -7.32421875e-02 -2.44140625e-01
  1.52343750e-01  1.20605469e-01  3.47656250e-01  7.42187500e-02
  7.03125000e-02  2.05078125e-01 -1.24023438e-01  1.80664062e-01
  1.02539062e-01 -1.17675781e-01  9.08203125e-02  9.71679688e-02
 -1.92871094e-02  1.95312500e-01 -1.08886719e-01  6.49414062e-02
  3.32031250e-01 -7.37304688e-02 -1.27929688e-01  4.96093750e-01
  1.16699219e-01 -6.54296875e-02  3.06640625e-01  1.28906250e-01
 -3.27148438e-02 -1.74804688e-01 -1.33789062e-01 -2.79296875e-01
 -1.66992188e-01  1.60156250e-01 -3.16406250e-01 -1.95312500e-02
  1.73339844e-02  2.63671875e-01  9.76562500e-02 -1.37939453e-02
 -1.36718750e-01 -2.14843750e-01  2.07519531e-02 -1.17675781e-01
  7.03125000e-02 -1.10351

In [3]:
# #编写函数并调用，通过对句子中的词向量求平均的方式获得句子的分布式表示，用于下文分类.
# #获得的训练集的向量矩阵存入train_vecs, 测试集的向量矩阵存入test_vecs.





def wordlist2vecs(corpus):
    res = []
    for wordlist in corpus:
        vecs = np.array([0]*300,dtype='float64')
        count = 0
        for word in wordlist:
            try:
                vecs += model[word]
            except:
                count += 1
        div = np.array([len(wordlist)-count]*300,dtype='float64')
        vecs = np.divide(vecs,div)
        res.append(vecs)

    return np.array(res)
    

train_vecs = wordlist2vecs(x_train)
test_vecs = wordlist2vecs(x_test)
# try:
#     model.get_index('a')
# except:
#     print('ok')


In [4]:
#向量缩放
from sklearn.preprocessing import scale
train_vecs=scale(train_vecs)
test_vecs=scale(test_vecs)

#分类
#from sklearn.metrics import roc_curve, auc
from sklearn.metrics import classification_report
from sklearn import  svm
clf = svm.SVC(kernel='linear', gamma=10)
clf.fit(train_vecs,y_train)
y_pred=clf.predict(test_vecs)
print(classification_report(y_test,y_pred))
print('Acc: %.2f' % clf.score(test_vecs, y_test))



              precision    recall  f1-score   support

           0       0.98      0.94      0.96       609
           1       0.96      0.99      0.97       809

    accuracy                           0.97      1418
   macro avg       0.97      0.96      0.97      1418
weighted avg       0.97      0.97      0.97      1418

Acc: 0.97


In [5]:

#using SLG to train classifier model, and assess
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import classification_report
lr=SGDClassifier(loss='log',penalty='l1')
lr.fit(train_vecs,y_train)
y_pred = lr.predict(test_vecs)
print(classification_report(y_test,y_pred))

print('Acc: %.2f' % lr.score(test_vecs, y_test))

c:\Python\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:173: FutureWarning: The loss 'log' was deprecated in v1.1 and will be removed in version 1.3. Use `loss='log_loss'` which is equivalent.
  warnings.warn(


              precision    recall  f1-score   support

           0       0.92      0.94      0.93       609
           1       0.95      0.94      0.95       809

    accuracy                           0.94      1418
   macro avg       0.94      0.94      0.94      1418
weighted avg       0.94      0.94      0.94      1418

Acc: 0.94
